<a href="https://colab.research.google.com/github/george-kan/fastai/blob/main/Mnist_full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#hide
!pip install -Uqq fastbook
import fastbook

     |████████████████████████████████| 727kB 17.3MB/s 
     |████████████████████████████████| 51kB 8.6MB/s 
     |████████████████████████████████| 1.2MB 50.7MB/s 
     |████████████████████████████████| 194kB 53.3MB/s 
     |████████████████████████████████| 61kB 8.8MB/s 


In [2]:
#hide
from fastai.vision.all import *
from fastbook import *

In [3]:
path = untar_data(URLs.MNIST)

In [4]:
(path/'training').ls()

(#10) [Path('/root/.fastai/data/mnist_png/training/5'),Path('/root/.fastai/data/mnist_png/training/6'),Path('/root/.fastai/data/mnist_png/training/2'),Path('/root/.fastai/data/mnist_png/training/3'),Path('/root/.fastai/data/mnist_png/training/0'),Path('/root/.fastai/data/mnist_png/training/1'),Path('/root/.fastai/data/mnist_png/training/8'),Path('/root/.fastai/data/mnist_png/training/7'),Path('/root/.fastai/data/mnist_png/training/9'),Path('/root/.fastai/data/mnist_png/training/4')]

In [6]:
folder = 'training/3'

(path/folder).ls()

(#6131) [Path('/root/.fastai/data/mnist_png/training/3/45979.png'),Path('/root/.fastai/data/mnist_png/training/3/39686.png'),Path('/root/.fastai/data/mnist_png/training/3/50007.png'),Path('/root/.fastai/data/mnist_png/training/3/54249.png'),Path('/root/.fastai/data/mnist_png/training/3/21359.png'),Path('/root/.fastai/data/mnist_png/training/3/16793.png'),Path('/root/.fastai/data/mnist_png/training/3/16924.png'),Path('/root/.fastai/data/mnist_png/training/3/1432.png'),Path('/root/.fastai/data/mnist_png/training/3/54339.png'),Path('/root/.fastai/data/mnist_png/training/3/52191.png')...]

In [14]:
(path/'testing').ls()

(#10) [Path('/root/.fastai/data/mnist_png/testing/5'),Path('/root/.fastai/data/mnist_png/testing/6'),Path('/root/.fastai/data/mnist_png/testing/2'),Path('/root/.fastai/data/mnist_png/testing/3'),Path('/root/.fastai/data/mnist_png/testing/0'),Path('/root/.fastai/data/mnist_png/testing/1'),Path('/root/.fastai/data/mnist_png/testing/8'),Path('/root/.fastai/data/mnist_png/testing/7'),Path('/root/.fastai/data/mnist_png/testing/9'),Path('/root/.fastai/data/mnist_png/testing/4')]

In [16]:
train_x = []
train_y = []
test_x = []
test_y = []
for digit in range(10):
    folder = 'training/' + str(digit)
    new_inp = [tensor(Image.open(x)) for x in (path/folder).ls()]
    train_x += new_inp
    train_y += [digit]*len(new_inp)
    test_folder = 'testing/' + str(digit)
    test_inp = [tensor(Image.open(x)) for x in (path/test_folder).ls()]
    test_x += test_inp
    test_y += [digit]*len(test_inp)
    

In [47]:
train_x_stacked = torch.stack(train_x).view(-1, 28*28).float()/255
train_y_stacked = tensor(train_y).view(-1, 1)
train_ds = list(zip(train_x_stacked, train_y_stacked))
train_dl = DataLoader(train_ds, batch_size= 256, shuffle=True)

test_x_stacked = torch.stack(test_x).view(-1, 28*28).float()/255
test_y_stacked = tensor(test_y).view(-1, 1)
test_ds = list(zip(test_x_stacked, test_y_stacked))
test_dl = DataLoader(test_ds, batch_size=256)

dls = DataLoaders(train_dl, test_dl)

train_x_stacked.shape, train_y_stacked.shape, test_x_stacked.shape, test_y_stacked.shape

(torch.Size([60000, 784]),
 torch.Size([60000, 1]),
 torch.Size([10000, 784]),
 torch.Size([10000, 1]))

In [9]:
train_x[2000]

tensor([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  71, 101, 237, 254, 255, 254,  92,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 177, 253, 253, 253, 253, 253, 219, 116,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  77, 146, 221, 253, 253, 253, 2

In [10]:
df = pd.DataFrame(torch.trunc(tensor(train_x[2000]).float()))
df.style.set_properties(**{'font-size':'6pt'}).background_gradient('Greys')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,71.000000,101.000000,237.000000,254.000000,255.000000,254.000000,92.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,177.000000,253.000000,253.000000,253.000000,253.000000,253.000000,219.000000,116.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,77.000000,146.000000,221.000000,253.000000,253.000000,253.000000,253.000000,253.000000,253.000000,207.000000,116.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,68.000000,176.000000,253.000000,253.000000,253.000000,253.000000,253.000000,253.000000,253.000000,253.000000,253.000000,142.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,89.000000,237.000000,253.000000,253.000000,253.000000,253.000000,242.000000,214.000000,214.000000,247.000000,253.000000,253.000000,203.000000,76.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,22.000000,139.000000,247.000000,253.000000,253.000000,253.000000,253.000000,253.000000,100.000000,0.000000,0.000000,216.000000,253.000000,253.000000,253.000000,99.000000,0.000000,0.000000,0.000000,0.000000


In [65]:
def batch_accuracy(x, y):
    preds = torch.argmax(x, 1)
    correct = preds == y
    return correct.float().mean()

simple_net = nn.Sequential(
            nn.Linear(28*28, 40),
            nn.ReLU(),
            nn.Linear(40, 40),
            nn.ReLU(),
            nn.Linear(40, 10)
)
loss = CrossEntropyLossFlat()
learn = Learner(dls, simple_net, opt_func = SGD, loss_func = loss, metrics = batch_accuracy)
learn.fit(40, 0.3)


epoch,train_loss,valid_loss,batch_accuracy,time
0,0.354481,0.379188,0.814179,00:01
1,0.231877,0.189817,0.866940,00:01
2,0.172891,0.191282,0.865955,00:01
3,0.145683,0.160573,0.876057,00:01
4,0.125333,0.203930,0.858171,00:01
5,0.112721,0.121421,0.886014,00:01
6,0.099954,0.108574,0.887913,00:01
7,0.091773,0.099209,0.892378,00:01
8,0.083011,0.119114,0.885196,00:01
9,0.072894,0.107500,0.889469,00:01


In [37]:
nn.Softmax(dim = 0)(simple_net(train_x_stacked[0]))

tensor([0.1123, 0.0809, 0.1052, 0.0939, 0.1033, 0.0765, 0.1111, 0.0943, 0.1205, 0.1021], grad_fn=<SoftmaxBackward>)

In [40]:
nn.Softmax(dim = 1)(simple_net(train_x_stacked[:2]))

tensor([[0.1123, 0.0809, 0.1052, 0.0939, 0.1033, 0.0765, 0.1111, 0.0943, 0.1205, 0.1021],
        [0.1052, 0.0765, 0.1148, 0.0964, 0.0972, 0.0744, 0.1103, 0.0983, 0.1339, 0.0930]], grad_fn=<SoftmaxBackward>)

In [44]:
torch.argmax(simple_net(train_x_stacked[:2]), 1)

tensor([8, 8])

In [53]:
batch_accuracy(simple_net(train_x_stacked[:2]), train_y_stacked[:2])
train_y_stacked[:2]
simple_net(train_x_stacked[:2])

tensor([[ 0.1082, -0.1231,  0.0778, -0.1022,  0.0530, -0.0264,  0.0766,  0.0732,  0.0385,  0.0532],
        [ 0.0583, -0.1157,  0.0564, -0.0323,  0.0883,  0.0379,  0.2472,  0.0184,  0.1086,  0.0130]], grad_fn=<AddmmBackward>)

In [54]:
??Learner